In [1]:
import pandas as pd
import numpy as np
import random
import heapq
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist

In [2]:
# Lokal
# Set Working Directory
%cd "C:\Users\Dennis\OneDrive\Dokumente\03_Master BAOR\05_Kurse\01_Business Analytics\04_Data Analytics Challenge"
ccdata = pd.read_csv('creditcard.csv')

C:\Users\Dennis\OneDrive\Dokumente\03_Master BAOR\05_Kurse\01_Business Analytics\04_Data Analytics Challenge


In [3]:
# Google Colab
# from google.colab import files
# uploaded = files.upload()
#ccdata = pd.read_csv('creditcard.csv')

In [4]:
# Small dataset
#ccdata = ccdata.iloc[:100000, :]

# Balance of dataset target values
display(ccdata.Class.value_counts())

# Drop feature 'Time'
ccdata.drop('Time', axis=1, inplace=True)

# Scaling feature 'Amount'
standard_scaler = StandardScaler()
ccdata['Amount'] = standard_scaler.fit_transform(ccdata[['Amount']])
ccdata.head(3)

0    284315
1       492
Name: Class, dtype: int64

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0


In [5]:
train_set, test_set = train_test_split(ccdata, train_size=0.9, random_state=1, stratify=ccdata['Class'])

print('{}\n'.format(train_set.Class.value_counts()))
print('Ratio of target value 1 in ccdata: {:.2f} %'.format(100*np.sum(ccdata['Class'] == 1)/len(ccdata['Class'])))
print('Ratio of target value 1 in train_set: {:.2f} %'.format(100*np.sum(train_set['Class'] == 1)/len(train_set['Class'])))
print('Ratio of target value 1 in test_set: {:.2f} %'.format(100*np.sum(test_set['Class'] == 1)/len(test_set['Class'])))

0    255883
1       443
Name: Class, dtype: int64

Ratio of target value 1 in ccdata: 0.17 %
Ratio of target value 1 in train_set: 0.17 %
Ratio of target value 1 in test_set: 0.17 %


In [6]:
#samples = ccdata
samples = train_set.reset_index()

# Create feature and target DF
samples_X = samples.iloc[:, 0:-1] #features
samples_y = samples.iloc[:,-1] #target variable

# Split it into minority set
Minority_sample = samples[samples['Class'] == 1] #minority set
Minority_sample_X = Minority_sample.iloc[:, 0:-1] #features of minority set as DF

# Create an array with only minority sample features and one with all sample features
Minority_X = np.array(Minority_sample_X) #features of Min set as Array
All_X = np.array(samples_X) #features of all samples

In [8]:
# Create distance matrix with cdist()-method
distance_matrix = pd.DataFrame(cdist(Minority_X, All_X), index=Minority_sample.index, columns=samples.index)
distance_matrix = np.where(distance_matrix == 0, 999999, distance_matrix).tolist()
#distance_df = pd.DataFrame(distance_matrix, index=Minority_sample.index, columns=samples.index)

# View matrix
#display(distance_df.head())

In [9]:
# Create empty matrix with n min instances as rows and k columns
nb_idx_matrix = np.zeros((Minority_X.shape[0], 5))

# Fill matrix with indices of k nearest neighbors for each min instance
for i in range(Minority_X.shape[0]): #für alle Min instances
    nb_idx_matrix[i] = list(map(distance_matrix[i].index, heapq.nsmallest(5, distance_matrix[i]))) #index des nächsten Nachbarn in Zeile der aktuellen Min instance

del(distance_matrix)
nb_idx_matrix[:12]

array([[ 69710.,  74920., 200407., 198809., 170162.],
       [ 19926.,  17648.,  26515., 132390., 148310.],
       [207558.,  65903., 203794., 245400.,  77382.],
       [ 89962., 158433.,  30116.,  58658., 154022.],
       [ 24109., 147617.,   7637., 185894.,   5684.],
       [ 93103., 241321., 119597., 136090., 145894.],
       [213774., 164126.,  12766.,  35491.,  60075.],
       [ 72757., 159835., 179245.,   7367., 112750.],
       [ 19681.,  80496., 155761., 190758.,  81031.],
       [119024.,  83176.,  27431., 135696.,  14174.],
       [ 42411.,  60427., 210902., 135495., 246277.],
       [119063., 119063., 203159., 249115., 245553.]])

In [10]:
# Create empty matrix with n min instances as rows and k columns
nb_y_matrix = np.zeros((Minority_X.shape[0], 5))

# Fill matrix with target value of k nearest neighbors for each min instance
for i, row in enumerate(nb_idx_matrix):
    for j, element in enumerate(row):
        nb_y_matrix[i,j] = samples_y[element]

nb_y_matrix[:12]

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1.]])

In [11]:
# Edit indices to integers (nested list comprehension)
#nb_idx_matrix = np.array([[int(neighbor) for neighbor in neighbors] for neighbors in nb_idx_matrix])

# Set values of maj instance neighbors to NAN
nb_y_matrix = np.where(nb_y_matrix==0, np.nan, nb_y_matrix)

# multiply both matricies to get only the indices of the neighbors from min class
q_nb_matrix = nb_y_matrix * nb_idx_matrix
q_nb_matrix[:12]

array([[    nan,     nan,     nan,     nan,     nan],
       [ 19926.,     nan,     nan,     nan,     nan],
       [207558.,  65903., 203794., 245400.,     nan],
       [    nan,     nan,     nan,     nan,     nan],
       [    nan,     nan,     nan,     nan,     nan],
       [    nan,     nan,     nan,     nan,     nan],
       [    nan,     nan,     nan,     nan,     nan],
       [    nan,     nan,     nan,     nan,     nan],
       [ 19681.,  80496., 155761., 190758.,  81031.],
       [    nan,     nan,     nan,     nan,     nan],
       [ 42411.,  60427.,     nan,     nan,     nan],
       [119063., 119063., 203159., 249115., 245553.]])

In [12]:
# Select only min instances with a min instance as nearest neighbor
q_min_idx = []
for index in range(len(q_nb_matrix)):
    if (np.isnan(q_nb_matrix[index][0]) == False):
        q_min_idx.append(index)

print('Number of qualified min instances: {}'.format(len(q_min_idx)))
print('Indices of first five qualified min instances: {}\n'.format(q_min_idx[:5]))
print('Indices of k-nearest neighbors of first five qualified min instances: \n{}'.format(q_nb_matrix[q_min_idx][:5]))

Number of qualified min instances: 135
Indices of first five qualified min instances: [1, 2, 8, 10, 11]

Indices of k-nearest neighbors of first five qualified min instances: 
[[ 19926.     nan     nan     nan     nan]
 [207558.  65903. 203794. 245400.     nan]
 [ 19681.  80496. 155761. 190758.  81031.]
 [ 42411.  60427.     nan     nan     nan]
 [119063. 119063. 203159. 249115. 245553.]]


In [13]:
# Select only the neighbors which are closer than the closest maj instance
q_nbs_dict = {}

for index, neighbors in enumerate(q_nb_matrix):
    if (np.isnan(q_nb_matrix[index]).any()):
        q_nbs = []
        for neighbor in neighbors:
            if np.isnan(neighbor):
                break
            else:
                q_nbs.append(int(neighbor))
        if (len(q_nbs) > 0): q_nbs_dict[index] = q_nbs
    else:
        q_nbs_dict[index] = [int(element) for element in neighbors]

# Dicitonary of qualified nearest neighbors indices for each qualified min instance
print('Indices of qualified nearest neighbors of first five min instances:')
for i in q_min_idx[:5]: print('{}: {}'.format(i,q_nbs_dict[i]))

Indices of qualified nearest neighbors of first five min instances:
1: [19926]
2: [207558, 65903, 203794, 245400]
8: [19681, 80496, 155761, 190758, 81031]
10: [42411, 60427]
11: [119063, 119063, 203159, 249115, 245553]


In [14]:
duplicates = np.array([(len(q_nbs_dict[i]) - len(set(q_nbs_dict[i]))) for i in q_nbs_dict.keys()])
print(duplicates)
print(len(q_min_idx) == len(duplicates) == len(q_nbs_dict))
print(q_nbs_dict[431])

duplicates_idx = np.where(duplicates == 2, 1, duplicates)

duplicates = list(q_nbs_dict.keys()) * duplicates_idx
print(duplicates[~np.isin(duplicates, 0)])

print(q_nbs_dict[11])
print(q_nbs_dict[349])
print(q_nbs_dict[427])
print(q_nbs_dict[431])


[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 1 0 0]
True
[203159, 203159, 246616, 6054, 119063]
[ 11 349 427 431]
[119063, 119063, 203159, 249115, 245553]
[246616, 246616, 6054, 6054, 119063]
[6054, 6054, 119063, 119063, 245553]
[203159, 203159, 246616, 6054, 119063]


In [16]:
# Algorithm 2: Adaptive neighbor instances selection
n = 10 #number of synthetic new instances per qualified Min instance
k = 5
synthetic = np.zeros((len(q_min_idx)*n, Minority_X.shape[1]), dtype=float) #Matrix(qualified Min instances * n x Number of Features)

# Algorithm 3: Procedure of ASN-SMOTE (Create new synthetic minority samples)
for min in q_min_idx:
    for new_sample in range(n):
        nn = random.randint(0, len(q_nbs_dict[min]) - 1) #Zufälliges Auswählen eines besten Nachbarn aus dem Set der qualifizierten Nachbarn
        print(min)
        print(q_nbs_dict[min][nn])
        dif = All_X[q_nbs_dict[min][nn]] - Minority_X[min] #Differenz des zufällig ausgewählten Nachbarn und der Min instance
        print(dif)
        gap = random.random() #Wert zw. 0 und 1
        synthetic[new_sample] = Minority_X[min] + gap * dif #Generierung einer neuen Min instance

1
19926
[-2.         -0.39977704 -0.29075251  1.33170411 -1.48054099  1.5072564
 -0.62877067  0.19335727 -0.31821314 -0.04825019 -0.8243259  -0.14603773
 -0.14555938 -0.36105571 -0.30210971 -1.21621636  0.59198712 -0.88305399
 -0.24937406 -1.99818034 -0.10754382  0.03484309 -0.19525062 -0.57148483
 -0.01172558  0.44199555 -0.29337147 -0.05337037  0.17886705 -0.44606648]
1
19926
[-2.         -0.39977704 -0.29075251  1.33170411 -1.48054099  1.5072564
 -0.62877067  0.19335727 -0.31821314 -0.04825019 -0.8243259  -0.14603773
 -0.14555938 -0.36105571 -0.30210971 -1.21621636  0.59198712 -0.88305399
 -0.24937406 -1.99818034 -0.10754382  0.03484309 -0.19525062 -0.57148483
 -0.01172558  0.44199555 -0.29337147 -0.05337037  0.17886705 -0.44606648]
1
19926
[-2.         -0.39977704 -0.29075251  1.33170411 -1.48054099  1.5072564
 -0.62877067  0.19335727 -0.31821314 -0.04825019 -0.8243259  -0.14603773
 -0.14555938 -0.36105571 -0.30210971 -1.21621636  0.59198712 -0.88305399
 -0.24937406 -1.99818034 -0.

In [17]:
# synthetic=synthetic[0:synthetic.shape[0]-,:]
labels = np.ones(synthetic.shape[0]) #Kennzeichnung der neuen synthetischen samples als Min instance mit class=1
synthetic = np.insert(synthetic, synthetic.shape[1], values=labels, axis=1)
examples = np.concatenate((samples, synthetic), axis=0) #Verbindung der samples mit den synthetischen samples

In [18]:
synthetic

array([[ 2.34632549e+05,  1.26132424e+00,  2.72680029e+00, ...,
         2.80486471e-01, -3.46872436e-01,  1.00000000e+00],
       [ 2.34632275e+05,  1.26132424e+00,  2.72680029e+00, ...,
         2.80486471e-01, -3.46872436e-01,  1.00000000e+00],
       [ 2.34632474e+05,  1.26132424e+00,  2.72680029e+00, ...,
         2.80486471e-01, -3.46872436e-01,  1.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])